3.1 Stacking One Rowset atop Another

In [ ]:
select 
    e.ENAME as ENAME_AND_DNAME,
    e.DEPTNO
from 
    dbo.EMP as e
where 
    e.DEPTNO = 10
union all 
select 
    '----------',
    null
union all 
select 
    d.DNAME,
    d.DEPTNO
from 
    dbo.DEPT as d

-- Union all will include diplicates if they exist. If yo want to filter out duplicates, use the UNION
select 
    e.DEPTNO
from 
    dbo.EMP as e
union
select 
    d.DEPTNO
from 
    dbo.DEPT as d


3.2 Combining Related Rows

In [ ]:
select 
    e.ENAME,
    d.LOC
from 
    dbo.EMP as e,
    dbo.DEPT as d
where 
    e.DEPTNO = 10
    and e.DEPTNO = d.DEPTNO

-- alternative version 
select 
    e.ENAME,
    d.LOC
from 
    dbo.EMP as e
inner join
    dbo.DEPT as d on e.DEPTNO = d.DEPTNO
where 
    e.DEPTNO = 10
    



3.3 Finding Rows in Common Between Two Tables

In [ ]:
drop view V;

In [ ]:
create view V 
as 
select 
    e.ENAME,
    e.JOB,
    e.SAL
from 
    dbo.EMP as e
where 
    e.JOB = 'CLERK'

In [ ]:
select * from V

select
    e.EMPNO,
    e.ENAME,
    e.JOB,
    e.SAL,
    e.DEPTNO
from
    dbo.V as v,
    dbo.EMP as e
where 
    v.ENAME = e.ENAME
    and v.JOB = e.JOB
    and v.SAL = e.SAL

-- alternative version
select
    e.EMPNO,
    e.ENAME,
    e.JOB,
    e.SAL,
    e.DEPTNO
from
    dbo.V as v
inner join 
    dbo.EMP as e on 
    (
        v.ENAME = e.ENAME
        and v.JOB = e.JOB
        and v.SAL = e.SAL
    )

3.4 Retrieving Values from One Table That Do Net Exist in Another

In [ ]:
select 
    d.DEPTNO
from 
    dbo.DEPT as d 
except 
select 
    e.DEPTNO 
from 
    dbo.EMP as e 

3.5 Retrieving Rows from One Table That Do Not Correspond to Rows in Another

In [ ]:
select 
    d.DEPTNO,
    d.DNAME,
    d.LOC
    -- ,e.*
from 
    dbo.DEPT as d 
left outer join 
    dbo.EMP as e on d.DEPTNO = e.DEPTNO
where 
    e.DEPTNO is null


3.6 Adding Joins to a Query Without Interfering with Other Joins

In [ ]:
drop table dbo.EMP_BONUS;

create table EMP_BONUS
(
    EMPNO int not null,
    RECEIVED datetime,
    [TYPE] int 
);

insert into dbo.EMP_BONUS values
    (7369, '14-MAR-2005', 1),
    (7900, '14-MAR-2005', 2),
    (7788, '14-MAR-2005', 3);

In [ ]:
select 
    e.ENAME,
    d.LOC,
    eb.RECEIVED
from 
    dbo.EMP as e
inner join 
    dbo.DEPT as d on e.DEPTNO = d.DEPTNO
left outer join 
    dbo.EMP_BONUS as eb on e.EMPNO = eb.EMPNO
order by 
    d.LOC asc 

-- alternative solution use scalar subquery
select 
    e.ENAME,
    d.LOC,
    (
        select
            eb.RECEIVED
        from 
            dbo.EMP_BONUS as eb
        where 
            eb.EMPNO = e.EMPNO
    ) as received
from 
    dbo.EMP as e,
    dbo.DEPT as d 
where 
    e.DEPTNO = d.DEPTNO
order by 
    d.LOC asc

3.7 Determining Whether Two Tables Have the Same Data (Cardinality and Values)

In [ ]:
drop view V;

In [ ]:
create view V 
as 
select 
    *
from 
    dbo.EMP as e
where 
    e.DEPTNO != 10
union all
select 
    *
from 
    dbo.EMP as e 
where 
    e.ENAME = 'WARD';

In [ ]:
select 
    *
from 
(
    select 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO,
        count(*) as count 
    from 
        dbo.EMP as e 
    group by 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO
) as e
where not exists -- compare view e with view v
(
    select 
        null
    from
    (
        select 
            v.EMPNO,
            v.ENAME,
            v.JOB,
            v.MGR,
            v.HIREDATE,
            v.SAL,
            v.COMM,
            v.DEPTNO,
            count(*) as count 
        from 
            dbo.V as v 
        group by 
            v.EMPNO,
            v.ENAME,
            v.JOB,
            v.MGR,
            v.HIREDATE,
            v.SAL,
            v.COMM,
            v.DEPTNO
    ) as v
    where 
        v.EMPNO = e.EMPNO
        and v.ENAME = e.ENAME
        and v.JOB = e.JOB
        and coalesce(v.MGR,0) = coalesce(e.MGR, 0)
        and v.HIREDATE = e.HIREDATE
        and v.SAL = e.SAL
        and coalesce(v.COMM,0) = coalesce(e.COMM, 0)
        and v.DEPTNO = e.DEPTNO    
        and v.[count] = e.[count]    
)
union all
select 
    *
from 
(
    select 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO,
        count(*) as count 
    from 
        dbo.V as v 
    group by 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO
) as v 
where not exists -- compare view v with view e
(
    select 
        null
    from
    (
        select 
            e.EMPNO,
            e.ENAME,
            e.JOB,
            e.MGR,
            e.HIREDATE,
            e.SAL,
            e.COMM,
            e.DEPTNO,
            count(*) as count 
        from 
            dbo.EMP as e 
        group by 
            e.EMPNO,
            e.ENAME,
            e.JOB,
            e.MGR,
            e.HIREDATE,
            e.SAL,
            e.COMM,
            e.DEPTNO
    ) as e
    where 
        v.EMPNO = e.EMPNO
        and v.ENAME = e.ENAME
        and v.JOB = e.JOB
        and coalesce(v.MGR,0) = coalesce(e.MGR, 0)
        and v.HIREDATE = e.HIREDATE
        and v.SAL = e.SAL
        and coalesce(v.COMM,0) = coalesce(e.COMM, 0)
        and v.DEPTNO = e.DEPTNO   
        and v.[count] = e.[count]
)

-- alternative solution using except 
(
    select 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO,
        count(*) as count 
    from 
        dbo.V as v 
    group by 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO
    except
    select 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO,
        count(*) as count 
    from 
        dbo.EMP as e 
    group by 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO
)
union all 
(
    select 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO,
        count(*) as count 
    from 
        dbo.EMP as e 
    group by 
        e.EMPNO,
        e.ENAME,
        e.JOB,
        e.MGR,
        e.HIREDATE,
        e.SAL,
        e.COMM,
        e.DEPTNO
    except
    select 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO,
        count(*) as count 
    from 
        dbo.V as v 
    group by 
        v.EMPNO,
        v.ENAME,
        v.JOB,
        v.MGR,
        v.HIREDATE,
        v.SAL,
        v.COMM,
        v.DEPTNO
)

3.8 Identifying and Avoiding Cartersian Products

In [ ]:
-- Cartersian Product query exemple 
select 
    e.ENAME,
    d.LOC
from 
    dbo.EMP as e,
    dbo.DEPT as d
where 
    e.DEPTNO = 10

-- solution 
select 
    e.ENAME,
    d.LOC
from 
    dbo.EMP as e,
    dbo.DEPT as d
where 
    e.DEPTNO = 10
    and e.DEPTNO = d.DEPTNO
    

3.9 Perfoming Joins When Using Aggregates

In [ ]:
drop table dbo.EMP_BONUS;

CREATE TABLE EMP_BONUS
(
    EMPNO int NOT NULL,
    RECEIVED DATETIME,
    [TYPE] int 
);

INSERT INTO EMP_BONUS VALUES
    (7934, '17-MAR-2005', 1),
    (7934, '15-FEB-2005', 2),
    (7839, '15-FEB-2005', 3),
    (7782, '15-FEB-2005', 1)

In [ ]:
select * from dbo.EMP_BONUS 

select 
    e.EMPNO
    ,e.ENAME
    ,e.SAL
    ,e.DEPTNO
    ,e.SAL * case
                when eb.[TYPE] = 1 then 0.1
                when eb.[TYPE] = 2 then 0.2
                else 0.3
            end as bonus
from 
    dbo.EMP e, 
    dbo.EMP_BONUS eb
where 
    e.EMPNO = eb.EMPNO
    and e.DEPTNO = 10


select 
    distinct tv.deptno 
    ,sum(tv.total_sal) as total_sal 
    ,tv.total_bonus
from 
(
    select 
        distinct e.EMPNO
        ,e.ENAME
        ,sum(e.SAL) as total_sal
        ,e.DEPTNO
        ,sum(e.SAL * case
                    when eb.[TYPE] = 1 then 0.1
                    when eb.[TYPE] = 2 then 0.2
                    else 0.3
                end) over (partition by e.DEPTNO) as total_bonus
    from 
        dbo.EMP e, 
        dbo.EMP_BONUS eb
    where 
        e.EMPNO = eb.EMPNO
        and e.DEPTNO = 10
    group by 
        e.EMPNO
        ,e.ENAME 
        ,e.DEPTNO
        ,e.SAL
        ,eb.[TYPE]
) as tv
group by 
    tv.DEPTNO
    ,tv.total_bonus


-- alternative solution without use over clause 
select 
    tv.DEPTNO
    ,sum(tv.SAL) as total_sal
    ,sum(tv.bonus) as total_bonus
from 
(
    select 
        e.EMPNO
        ,e.ENAME
        ,e.SAL 
        ,e.DEPTNO
        ,sum(e.SAL * case
                    when eb.[TYPE] = 1 then 0.1
                    when eb.[TYPE] = 2 then 0.2
                    else 0.3
                end) as bonus
    from 
        dbo.EMP e, 
        dbo.EMP_BONUS eb
    where 
        e.EMPNO = eb.EMPNO
        and e.DEPTNO = 10
    group by 
        e.EMPNO
        ,e.ENAME 
        ,e.SAL
        ,e.DEPTNO
) as tv
group by
    tv.DEPTNO

3.10 Perfoming Outer Joins When Using Aggregates

In [ ]:
drop table dbo.EMP_BONUS;

CREATE TABLE EMP_BONUS
(
    EMPNO int NOT NULL,
    RECEIVED DATETIME,
    [TYPE] int 
);

INSERT INTO EMP_BONUS VALUES
    (7934, '17-MAR-2005', 1),
    (7934, '15-FEB-2005', 2)

In [ ]:
select * from dbo.EMP_BONUS

select 
    tv.DEPTNO
    ,sum(tv.SAL) as total_sal
    ,sum(tv.bonus) as total_bonus
from 
(
    select 
        e.EMPNO
        ,e.ENAME
        ,e.SAL 
        ,e.DEPTNO
        ,sum(e.SAL * case
                    when eb.[TYPE] = 1 then 0.1
                    when eb.[TYPE] = 2 then 0.2
                    when eb.[TYPE] = 3 then 0.3
                    else 0.0
                end) as bonus
    from 
        dbo.EMP e
    left outer join 
        dbo.EMP_BONUS eb on e.EMPNO = eb.EMPNO
    where 
        e.DEPTNO = 10
    group by 
        e.EMPNO
        ,e.ENAME 
        ,e.SAL
        ,e.DEPTNO
) as tv
group by
    tv.DEPTNO

3.11 Returning Missing Data from Multiple Tables

In [ ]:
select 
    d.DEPTNO
    ,d.DNAME
    ,e.ENAME
from 
    dbo.DEPT as d
full outer join 
    dbo.EMP as e on d.DEPTNO = e.DEPTNO

-- alternative solution using union and left/rigth join
select 
    d.DEPTNO
    ,d.DNAME
    ,e.ENAME
from 
    dbo.DEPT as d
left outer join 
    dbo.EMP as e on d.DEPTNO = e.DEPTNO
union 
select 
    d.DEPTNO
    ,d.DNAME
    ,e.ENAME
from 
    dbo.DEPT as d
right outer join 
    dbo.EMP as e on d.DEPTNO = e.DEPTNO



3.12 Usin NULLs in Operations and Comparisons

In [10]:
select 
    ename, comm
from 
    dbo.EMP as e
where coalesce(comm, 0) < (select comm from dbo.EMP where ename = 'WARD');


(13 rows affected)

Total execution time: 00:00:00.011

ename,comm
SMITH,NULL
ALLEN,300
JONES,NULL
BLAKE,NULL
CLARK,NULL
SCOTT,NULL
KING,NULL
TURNER,0
ADAMS,NULL
JAMES,NULL
